# Install Required Software

In [1]:
#Mambaforge
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
!bash Mambaforge-$(uname)-$(uname -m).sh -b -u -p $HOME/mambaforge

#Trimmomatic, Fastqc, bowtie, samtools, rsem
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda trimmomatic fastqc bowtie samtools rsem multiqc

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 90.4M  100 90.4M    0     0  21.9M      0  0:00:04  0:00:04 --:--:-- 26.7M
PREFIX=/home/jupyter/mambaforge
Unpacking payload ...
Extracting python-3.10.6-ha86cf86_0_cpython.tar.bz2
Extracting _libgcc_mutex-0.1-conda_forge.tar.bz2
Extracting ca-certificates-2022.9.24-ha878542_0.tar.bz2
Extracting ld_impl_linux-64-2.39-hc81fddc_0.tar.bz2
Extracting libstdcxx-ng-12.2.0-h46fd767_19.tar.bz2
Extracting pybind11-abi-4-hd8ed1ab_3.tar.bz2
Extracting tzdata-2022f-h191b570_0.tar.bz2
Extracting libgomp-12.2.0-h65d4601_19.tar.bz2
Extracting _openmp_mutex-4.5-2_gnu.tar.bz2
Extracting libgcc-ng-12.2.0-h65d4601_19.tar.bz2
Extracting bzip2-1.0.8-h7f98852_4.tar.bz2
Extracting c-ares-1.18.1

# Download Files

We need to create the folders we will use, as well as download the fastq and reference files that will be used for analysis. 

In this particular case, this notebook and the next notebook will only be looking at the 24 hour post injection IAV and HBSS 'spotless' zebrafish samples.

The 24 hour run IDs are :


SA161918 (HBSS)
SA161919 (HBSS)
SA161920 (HBSS)
SA161921 (HBSS)
SA161922 (IAV)
SA161923 (IAV)
SA161924 (IAV)
SA161878 (IAV)


In [2]:
## Create Directories to hold files
!mkdir -p data
!mkdir -p data/raw_fastq
!mkdir -p data/trimmed
!mkdir -p data/fastqc
!mkdir -p data/multiqc
!mkdir -p data/reference
!mkdir -p data/rsem

## Download Fastq files

# Download 24 hr runs from the Google bucket folder
# the * wild card will let us download the forward and reverse
# fastq files in one command

!gsutil cp gs://hon_350/fastq/SE8159_SA161918* data/raw_fastq
!gsutil cp gs://hon_350/fastq/SE8159_SA161919* data/raw_fastq
!gsutil cp gs://hon_350/fastq/SE8159_SA161920* data/raw_fastq
!gsutil cp gs://hon_350/fastq/SE8159_SA161921* data/raw_fastq
!gsutil cp gs://hon_350/fastq/SE8159_SA161922* data/raw_fastq
!gsutil cp gs://hon_350/fastq/SE8159_SA161923* data/raw_fastq
!gsutil cp gs://hon_350/fastq/SE8159_SA161924* data/raw_fastq
!gsutil cp gs://hon_350/fastq/SE8161_SA161878* data/raw_fastq

## Adapter sequences file for trimmomatic
!gsutil cp gs://hon_350/reference/TruSeq3-PE.fa data/reference/TruSeq3-PE.fa

## (Optional) Download reference files for building transcriptome index
#!gsutil cp gs://hon_350/referebce/GRCz11.dna_sm.toplevel.fa.gz data/reference/GRCz11.dna_sm.toplevel.fa.gz
#!gsutil cp gs://hon_350/referebce/GRCz11.108.gtf.gz data/reference/GRCz11.108.gtf.gz
#!echo 'unzipping...'
#!gzip -d data/reference/GRCz11.dna_sm.toplevel.fa.gz
#!gzip -d data/reference/GRCz11.107.gtf.gz
#!echo 'finished unzipping.'

## Download and unzip premade index files
!gsutil cp gs://hon_350/reference/rsem_grcz11.108_transcriptome.tar.gz data/reference/rsem_grcz11.108_transcriptome.tar.gz
!echo 'unzipping...'
!tar -xzvf data/reference/rsem_grcz11.108_transcriptome.tar.gz
!rm data/reference/rsem_grcz11.108_transcriptome.tar.gz
!echo 'finished unzipping.'

Copying gs://hon_350/fastq/SE8161_SA161878_S1_L007_R1_001.fastq.gz...
Copying gs://hon_350/fastq/SE8161_SA161878_S1_L007_R2_001.fastq.gz...           
\ [2 files][  2.6 GiB/  2.6 GiB]  298.5 MiB/s                                   
Operation completed over 2 objects/2.6 GiB.                                      
Copying gs://hon_350/fastq/SE8159_SA161918_S9_L007_R1_001.fastq.gz...
Copying gs://hon_350/fastq/SE8159_SA161918_S9_L007_R2_001.fastq.gz...           
\ [2 files][  2.6 GiB/  2.6 GiB]  142.5 MiB/s                                   
Operation completed over 2 objects/2.6 GiB.                                      
Copying gs://hon_350/fastq/SE8159_SA161919_S10_L007_R1_001.fastq.gz...
Copying gs://hon_350/fastq/SE8159_SA161919_S10_L007_R2_001.fastq.gz...          
/ [2 files][  2.6 GiB/  2.6 GiB]  122.8 MiB/s                                   
Operation completed over 2 objects/2.6 GiB.                                      
Copying gs://hon_350/fastq/SE8159_SA161920_S11_L007_R1_00

# Trimmomatic

Trimmomatic is used to trim fastq files to remove adapters, and low quality score bases in reads.

A nice guide on trimmomatic parameters: https://datacarpentry.org/wrangling-genomics/03-trimming/index.html 

Trimmomatic works on one paired or single end run at a time.

However we have dozens of paired-end reads. 

To speed things up, this example 'pipes in' a list of run ids into trimmomatic to run sequentially.

In [3]:
# Run trimmomatic
# trimmomatic works on one paired-end reads file
# at a time. an example might look like this:

#!trimmomatic PE -threads 2 data/raw_fastq/trunc_SE6151_SA60191_S8_L004_R1_001.fastq.gz data/raw_fastq/trunc_SE6151_SA60191_S8_L004_R2_001.fastq.gz data/trimmed/SA60191_1_trimmed.fastq.gz data/trimmed/SA60191_1_orphans.fastq.gz data/trimmed/SA60191_2_trimmed.fastq.gz data/trimmed/SA60191_2_orphans.fastq.gz ILLUMINACLIP:data/reference/TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

# for the sake of this example a list is 
# created using filenames in the fastq
# directory, and those names are piped in
# iteratively to the command function

# a list of the runs is made
!ls data/raw_fastq/*R1* |  sed 's|data/raw_fastq/||g' | sed 's|1_001.fastq.gz||g' > run_ids.txt

#what the list looks like
!cat run_ids.txt
!echo ' '

#the list is piped into trimmomatic
!cat run_ids.txt |  xargs -I {} trimmomatic PE -threads 16 data/raw_fastq/{}1_001.fastq.gz data/raw_fastq/{}2_001.fastq.gz data/trimmed/{}1_trimmed.fastq.gz data/trimmed/{}1_orphans.fastq.gz data/trimmed/{}2_trimmed.fastq.gz data/trimmed/{}2_orphans.fastq.gz ILLUMINACLIP:data/reference/TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

#may not be necessary,
#but just to save space, delete raw fastq files after
!rm -r data/raw_fastq

SE8159_SA161918_S9_L007_R
SE8159_SA161919_S10_L007_R
SE8159_SA161920_S11_L007_R
SE8159_SA161921_S12_L007_R
SE8159_SA161922_S13_L007_R
SE8159_SA161923_S14_L007_R
SE8159_SA161924_S15_L007_R
SE8161_SA161878_S1_L007_R
 
TrimmomaticPE: Started with arguments:
 -threads 2 data/raw_fastq/SE8159_SA161918_S9_L007_R1_001.fastq.gz data/raw_fastq/SE8159_SA161918_S9_L007_R2_001.fastq.gz data/trimmed/SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz data/trimmed/SE8159_SA161918_S9_L007_R1_orphans.fastq.gz data/trimmed/SE8159_SA161918_S9_L007_R2_trimmed.fastq.gz data/trimmed/SE8159_SA161918_S9_L007_R2_orphans.fastq.gz ILLUMINACLIP:data/reference/TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 27207460 Both Surviving: 268012

# Fastqc

Fastqc is a tool to inspect the quality of reads in fastq files.

In [4]:
# Run fastqc
!fastqc -o data/fastqc data/trimmed/*_trimmed.fastq.gz

# Example of in-window display of fastqc output file (SA60184).
from IPython.display import IFrame
IFrame(src='./data/fastqc/SE8159_SA161918_S9_L007_R1_trimmed_fastqc.html', width=800, height=600)


Started analysis of SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz
Approx 5% complete for SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz
Approx 10% complete for SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz
Approx 15% complete for SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz
Approx 20% complete for SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz
Approx 25% complete for SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz
Approx 30% complete for SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz
Approx 35% complete for SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz
Approx 40% complete for SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz
Approx 45% complete for SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz
Approx 50% complete for SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz
Approx 55% complete for SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz
Approx 60% complete for SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz
Approx 65% complete for SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz
Approx 70% complete for SE8159_SA161918_S9_L007_R1_tr

In [5]:
!multiqc -f data/fastqc -o data/multiqc

import pandas as pd
dframe = pd.read_csv("data/multiqc/multiqc_data/multiqc_fastqc.txt", sep='\t')
display(dframe)


  /// ]8;id=303068;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.13

|           multiqc | Search path : /home/jupyter/Example_Whim_Notebook/data/fastqc
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 32/32  0m  
|            fastqc | Found 16 reports
|           multiqc | Compressing plot data
|           multiqc | Report      : data/multiqc/multiqc_report.html
|           multiqc | Data        : data/multiqc/multiqc_data
|           multiqc | MultiQC complete


,Sample,Filename,File type,Encoding,Total Sequences,Sequences flagged as poor quality,Sequence length,%GC,total_deduplicated_percentage,avg_sequence_length,...,per_base_sequence_quality,per_tile_sequence_quality,per_sequence_quality_scores,per_base_sequence_content,per_sequence_gc_content,per_base_n_content,sequence_length_distribution,sequence_duplication_levels,overrepresented_sequences,adapter_content
0,SE8159_SA161918_S9_L007_R1,SE8159_SA161918_S9_L007_R1_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,26801236.0,0.0,36-151,50.0,86.570195,114.372489,...,pass,pass,pass,fail,fail,pass,warn,pass,warn,pass
1,SE8159_SA161918_S9_L007_R2,SE8159_SA161918_S9_L007_R2_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,26801236.0,0.0,36-151,51.0,82.679703,114.439963,...,pass,warn,pass,warn,fail,pass,warn,pass,warn,pass
2,SE8159_SA161919_S10_L007_R1,SE8159_SA161919_S10_L007_R1_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,25561141.0,0.0,36-151,49.0,88.268664,116.899008,...,pass,pass,pass,fail,fail,pass,warn,pass,warn,pass
3,SE8159_SA161919_S10_L007_R2,SE8159_SA161919_S10_L007_R2_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,25561141.0,0.0,36-151,49.0,84.683425,116.964957,...,pass,warn,pass,warn,fail,pass,warn,pass,warn,pass
4,SE8159_SA161920_S11_L007_R1,SE8159_SA161920_S11_L007_R1_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,24640681.0,0.0,36-151,49.0,88.817167,118.067488,...,pass,pass,pass,fail,fail,pass,warn,pass,warn,pass
5,SE8159_SA161920_S11_L007_R2,SE8159_SA161920_S11_L007_R2_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,24640681.0,0.0,36-151,49.0,84.407738,118.124288,...,pass,warn,pass,warn,fail,pass,warn,pass,warn,pass
6,SE8159_SA161921_S12_L007_R1,SE8159_SA161921_S12_L007_R1_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,24006271.0,0.0,36-151,51.0,87.490356,114.099170,...,pass,pass,pass,fail,fail,pass,warn,pass,warn,pass
7,SE8159_SA161921_S12_L007_R2,SE8159_SA161921_S12_L007_R2_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,24006271.0,0.0,36-151,51.0,83.541225,114.161399,...,pass,warn,pass,warn,fail,pass,warn,pass,warn,pass
8,SE8159_SA161922_S13_L007_R1,SE8159_SA161922_S13_L007_R1_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,27484431.0,0.0,36-151,48.0,87.501492,112.275292,...,pass,pass,pass,fail,fail,pass,warn,pass,warn,pass
9,SE8159_SA161922_S13_L007_R2,SE8159_SA161922_S13_L007_R2_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,27484431.0,0.0,36-151,49.0,85.277345,112.344603,...,pass,warn,pass,warn,fail,pass,warn,pass,warn,pass


# Rsem

In order to map reads to genes, and quantify read counts, an index must be created. In this case, we are using rsem for index creation and quantifying read counts.

The output and error log for this software is too large to display in the cell below, so is stored in a text log file 'rsem/rsem_log.txt' instead. 

For the example truncated set this can finish within a few minutes, but with the actual data set this will take around eight hours to fully run.

In [6]:
## Build transcriptome reference

#!rsem-prepare-reference --gtf data/reference/GRCz11.108.gtf --bowtie -p 8 data/reference/GRCz11.dna_sm.toplevel.fa data/reference/zebrafish_rsem/GRCz11_108 

In [ ]:
# to make writing commands easier,
# again we will make a list and 'pipe' the list
# into rsem.
# first make a list
!ls data/trimmed/*R1*trimmed* |  sed 's|data/trimmed/||g' | sed 's|.fastq.gz||g' > trimmed_ids.txt
# then run rsem
!echo 'running calculate counts...'
!cat trimmed_ids.txt |  xargs -I {} rsem-calculate-expression -p 16 --forward-prob 0 data/trimmed/{}.fastq.gz data/reference/zebrafish_rsem/GRCz11_108 data/rsem/{} >data/rsem/rsem_log.txt 2>&1
!echo 'finished'

running calculate counts...


# Read count file outputs

Rsem outputs readcount files (XXXXX.genes.results) which are tables that contain various information, including readcounts in transcripts per kilobase million (TPM) and fragments per kilobase of exon per million mapped fragments (FPKM) format.

In [9]:
# Example of read count table output
# (sorted by top 10 expressed genes)
!head -1 $(ls data/rsem/*.genes.results | head -n1) | column -t
!sort -nrk 6 $(ls data/rsem/*.genes.results | head -n1) |  column -t | head -10  | column -t

gene_id  transcript_id(s)  length  effective_length  expected_count  TPM  FPKM
ENSDARG00000087732  ENSDART00000122203  298.00   184.61   426598.37  131833.23  185369.87
ENSDARG00000114494  ENSDART00000187218  165.00   51.61    106601.97  117835.88  165688.29
ENSDARG00000110859  ENSDART00000193467  165.00   51.61    33489.10   37018.24   52051.11
ENSDARG00000109433  ENSDART00000180315  165.00   51.61    33489.10   37018.24   52051.11
ENSDARG00000083063  ENSDART00000118247  165.00   51.61    33489.10   37018.24   52051.11
ENSDARG00000089382  ENSDART00000126982  1212.00  1098.61  711006.00  36922.86   51917.01
ENSDARG00000114910  ENSDART00000183213  312.00   198.61   85085.32   24440.77   34366.01
ENSDARG00000081580  ENSDART00000116507  72.00    2.68     1090.00    23190.21   32607.61
ENSDARG00000081270  ENSDART00000117461  322.00   208.61   82572.80   22582.05   31752.48
ENSDARG00000082753  ENSDART00000115891  952.00   838.61   328630.40  22356.98   31436.00


# Upload files to bucket
In order for the files to be shared and downloaded, we need to upload them to the google bucket.

In [10]:
## Upload to bucket
!gsutil cp data/rsem/*.genes.results gs://hon_350/24hours_results/

Copying file://data/rsem/SE8159_SA161918_S9_L007_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
Copying file://data/rsem/SE8159_SA161919_S10_L007_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
Copying file://data/rsem/SE8159_SA161920_S11_L007_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
Copying file://data/rsem/SE8159_SA161921_S12_L007_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
\ [4 files][ 10.6 MiB/ 10.6 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://data/rsem/SE8159_SA161922_S13_L007_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
Copying file://data/rsem/SE8159_SA161923_S14_L007_R1_trimmed.genes.results [Co